In [0]:
#based on slides by K. Erickson

from __future__ import print_function

!pip install cobra
import cobra.test
from cobra import Model, Reaction, Metabolite


In [0]:
import cobra.test
#Load the model for genome scale E. coli iJO1366
model = cobra.test.create_test_model("ecoli")
#this model has 1366 genes, 2251 metabolic reactions, and 1136 unique metabolites

In [0]:
model.reactions[47].id
'EX_ade_e'
model.reactions[47].lower_bound
0.0
model.reactions[47].reaction
'ade_e --> '
model.objective

In [0]:
#Set constraints for aerobic growth in glucose minimal media
model.reactions.get_by_id("EX_glc__D_e").lower_bound= -10
model.reactions.get_by_id("EX_o2_e").lower_bound = -15

#Solve
solution = model.optimize()
#Output solution
print('Growth Rate: '+str(solution.objective_value)+' 1/h')
# Output more information
model.summary()

Growth Rate: 0.899217260405845 1/h


In [0]:
#Add crtEBI pathway for lycopene production

#New metabolites: ggpp_c, phyto_c, lyco_c
from cobra import Metabolite
coa_c = model.metabolites.get_by_id( 'coa_c')
ipdp_c = model.metabolites.get_by_id( 'ipdp_c')
frdp_c = model.metabolites.get_by_id( 'frdp_c')
ppi_c = model.metabolites.get_by_id( 'ppi_c')
nadp_c = model.metabolites.get_by_id( 'nadp_c')
nadph_c = model.metabolites.get_by_id( 'nadph_c')
#Create new metabolites
ggpp_c = Metabolite( 'ggpp_c', formula='C20H36O7P2', name='Geranylgeranyl Pyrophospate', compartment ='c')
phyto_c = Metabolite( 'phyto_c', formula='C40H64', name='Phytoene', compartment ='c')
lyco_c = Metabolite( 'lyco_c', formula='C40H56', name='Lycopene', compartment ='c')

In [0]:
#New reactions: CRTE, CRTB, CRTI, LYCO-dem
from cobra import Reaction
#add CRTE:
reaction1 = Reaction('CRTE')
reaction1.name = 'Geranylgeranyl diphosphate (GGPP) synthase'
reaction1.subsystem = 'Lycopene biosynthesis'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
reaction1.add_metabolites({ipdp_c: -1.0, frdp_c: -1.0, ggpp_c: 1.0, ppi_c: 1.0})
model.add_reaction(reaction1)
#add CRTB:
reaction2 = Reaction('CRTB')
reaction2.name = 'Phytoene synthase'
reaction2.subsystem = 'Lycopene biosynthesis'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
reaction2.add_metabolites({ggpp_c: -2.0, phyto_c: 1.0, ppi_c: 1.0})
model.add_reaction(reaction2)
#add CRTI:
reaction3 = Reaction('CRTI')
reaction3.name = 'Phytoene desaturase'
reaction3.subsystem = 'Lycopene biosynthesis'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({phyto_c: -1.0, nadp_c: -8.0, lyco_c: 1.0, nadph_c: 8.0})
model.add_reaction(reaction3)
#add LYCO-dem:
reaction4 = Reaction('LYCO-dem')
reaction4.name = 'Lycopene demand'
reaction4.subsystem = 'Lycopene biosynthesis'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({lyco_c: -1.0})
model.add_reaction(reaction4)

In [0]:
#FVA
from cobra.flux_analysis import flux_variability_analysis
reactions_OE = [model.reactions.DXPS, model.reactions.IPDDI, model.reactions.MECDPS, model.reactions.MEPCT]
fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print (fva)


         minimum   maximum
DXPS    0.002294  1.567733
IPDDI  -1.175787  0.391585
MECDPS  0.001933  1.567372
MEPCT   0.001933  1.567372


In [0]:
#Set the objective to Biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0

#Solve

solution=model.optimize() #solution is stored at model.solution
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))

Growth Rate (1/h): 0.9009127876086938
Lycopene Production Rate (mmol/gdcw/h): 0.0
Lycopene Yield (mol/mol glucose): 0.0


In [0]:
#Set the objective to lycopene
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 1.0

#This is the theoretical maximum lycopene yield

solution=model.optimize()

print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))


Growth Rate (1/h): 0.0
Lycopene Production Rate (mmol/gdcw/h): 1.101916572717023
Lycopene Yield (mol/mol glucose): 0.1101916572717023


In [0]:
DXPSlowerbound = model.reactions.get_by_id('DXPS').lower_bound
IPDDIlowerbound = model.reactions.get_by_id('IPDDI').lower_bound
MECDPSlowerbound = model.reactions.get_by_id('MECDPS').lower_bound
MEPCTlowerbound = model.reactions.get_by_id('MEPCT').lower_bound

#for overexpression set lower bounds to above maximum

#         minimum   maximum
#DXPS    0.002294  1.567733     -> set lower bound to 2
#IPDDI  -1.175787  0.391585     -> set lower bound to 0.5
#MECDPS  0.001933  1.567372     -> set lower bound to 2
#MEPCT   0.001933  1.567372     -> set lower bound to 2


#Overexpress dxs, idi, ispFD
model.reactions.get_by_id('DXPS').lower_bound = 2
model.reactions.get_by_id('IPDDI').lower_bound = 0.5
model.reactions.get_by_id('MECDPS').lower_bound = 2
model.reactions.get_by_id('MEPCT').lower_bound = 2


In [0]:
#Set the objective to Biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0

#Solve to get rates with overexpression

solution=model.optimize()
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))

Growth Rate (1/h): 0.7647818319530112
Lycopene Production Rate (mmol/gdcw/h): 0.24963787580257024
Lycopene Yield (mol/mol glucose): 0.024963787580257024


In [0]:
#Knockout genes gdhA, aceE, ytjC(gpmB), fdhF (yjjD, rssB, yjfP aren't in model)
model.genes.b1761.knock_out() # gdhA
model.genes.b0114.knock_out() # aceA
model.genes.b4395.knock_out() # ytjC
model.genes.b4079.knock_out() # fdhF
#undo the overexpression
model.reactions.get_by_id('DXPS').lower_bound = DXPSlowerbound
model.reactions.get_by_id('IPDDI').lower_bound = IPDDIlowerbound
model.reactions.get_by_id('MECDPS').lower_bound = MECDPSlowerbound
model.reactions.get_by_id('MEPCT').lower_bound = MEPCTlowerbound

In [0]:
#Set the objective to Biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0

#Solve to get rates with knockouts

solution=model.optimize()
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))

Growth Rate (1/h): 0.7962601526733222
Lycopene Production Rate (mmol/gdcw/h): 0.0
Lycopene Yield (mol/mol glucose): 0.0


In [0]:
#Knockout genes gdhA, aceE, ytjC(gpmB), fdhF (yjjD, rssB, yjfP aren't in model)
model.genes.b1761.knock_out() # gdhA
model.genes.b0114.knock_out() # aceA
model.genes.b4395.knock_out() # ytjC
model.genes.b4079.knock_out() # fdhF

#Overexpress dxs, idi, ispFD
model.reactions.get_by_id('DXPS').lower_bound = 2
model.reactions.get_by_id('IPDDI').lower_bound = 0.5
model.reactions.get_by_id('MECDPS').lower_bound = 2
model.reactions.get_by_id('MEPCT').lower_bound = 2


In [0]:
#Set the objective to Biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0

#Solve to get rates with overexpression and knockouts

solution=model.optimize()
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))

Growth Rate (1/h): 0.7085755330435601
Lycopene Production Rate (mmol/gdcw/h): 0.2496644894851038
Lycopene Yield (mol/mol glucose): 0.02496644894851038
